In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scipy.optimize import fsolve, minimize
from ipywidgets import FloatSlider, FloatText, Button, HBox, VBox, Output, Layout

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Constants
K = 8.617e-5  # Boltzmann constant (eV/K)
T = 300       # Temperature (K)

# Load J-V Data
file_path = os.path.join("Diode_analysis", "Book2.xlsx")
sheet_name = "Sheet1"
data = pd.read_excel(file_path, sheet_name=sheet_name)
data.columns = data.columns.str.strip()

V_exp = data["Voltage (V)"].values
J_exp = data["Current Density (mA/cm_2)"].values

# Define equation
def diode_eq(J, V, J0, Rs, n, Rsh):
    return J0 * (np.exp((V - J * Rs) / (n * K * T)) - 1) + (V - J * Rs) / Rsh - J

# Solve J-V curve
def solve_jv(J0, Rs, n, Rsh, V, J_init_pos, J_init_neg):
    return np.array([fsolve(diode_eq, J_init_pos if V_i > 0 else J_init_neg, args=(V_i, J0, Rs, n, Rsh))[0] for V_i in V])

# Define cost function
def cost_function(params, J_init_pos, J_init_neg):
    J0, Rs, n, Rsh = params
    J_calc = solve_jv(J0, Rs, n, Rsh, V_exp, J_init_pos, J_init_neg)
    return np.mean((J_calc - J_exp) ** 2)

# **🔧 Initial Estimates**
Rs_initial = (V_exp[-1] - V_exp[-2]) / (J_exp[-1] - J_exp[-2]) 
J0_initial = 1e-7
n_initial = 1.9
Rsh_initial = 10 

initial_params = [J0_initial, Rs_initial, n_initial, Rsh_initial]
bounds = [(1e-12, 1e-3), (0, 10), (1, 2), (100, 100000)]
J_initial_positive = 0.001
J_initial_negative = -0.0011

# **Optimize Parameters**
status_output = Output()
with status_output:
    print("Optimizing parameters... Please wait.")
result = minimize(cost_function, initial_params, args=(J_initial_positive, J_initial_negative), bounds=bounds, method='L-BFGS-B')
J0_opt, Rs_opt, n_opt, Rsh_opt = result.x  

# Convert to Ω·cm² for UI display
Rs_display = Rs_opt * 1000
Rsh_display = Rsh_opt * 1000

with status_output:
    status_output.clear_output()
    print(f"Optimization Complete!\nJ0: {J0_opt:.4e} mA/cm², Rs: {Rs_display:.4f} Ω·cm², n: {n_opt:.4f}, Rsh: {Rsh_display:.4f} Ω·cm²")

# **📏 UI Styling**
slider_layout = Layout(width='600px')
text_layout = Layout(width='100px')

# **📏 Create Sliders with Display in Ω·cm²**
def create_slider_text(description, min_val, max_val, step, value, unit):
    slider = FloatSlider(min=min_val, max=max_val, step=step, value=value, description=f"{description} ({unit})", layout=slider_layout)
    text = FloatText(value=value, layout=text_layout)
    return slider, text

J_init_pos_slider, J_init_pos_text = create_slider_text("J init (V>0)", 0, 10000, 0.001, J_initial_positive, "mA/cm²")
J_init_neg_slider, J_init_neg_text = create_slider_text("J init (V<0)", -0.1, 0, 0.0001, J_initial_negative, "mA/cm²")

J0_slider, J0_text = create_slider_text("J0", 1e-12, 1e-3, 1e-8, J0_opt, "mA/cm²")
Rs_slider, Rs_text = create_slider_text("Rs", 0, 100, 1, Rs_display, "Ω·cm²")  # Display in Ω·cm²
n_slider, n_text = create_slider_text("n", 1, 3, 0.001, n_opt, "")
Rsh_slider, Rsh_text = create_slider_text("Rsh", 100, 1e8, 100, Rsh_display, "Ω·cm²")  # Display in Ω·cm²


# **📉 Plot Output**
plot_output = Output()

def update_plot(_):
    with plot_output:
        plot_output.clear_output(wait=True)

        J0, Rs_kOhm, n, Rsh_kOhm = J0_slider.value, Rs_slider.value / 1000, n_slider.value, Rsh_slider.value / 1000  # Convert back to kΩ·cm² for solver

        J_calc = solve_jv(J0, Rs_kOhm, n, Rsh_kOhm, V_exp, J_init_pos_slider.value, J_init_neg_slider.value)

        plt.figure(figsize=(8, 5))
        plt.semilogy(V_exp, np.abs(J_exp), 'b', label="Experimental Data")
        plt.semilogy(V_exp, np.abs(J_calc), 'r-', label="Fitted Curve")
        plt.xlabel("Voltage (V)")
        plt.ylabel("|Current Density| (mA/cm²)")
        plt.title("J-V Curve (Log Scale)")
        plt.grid(True, which="both", ls="--")
        plt.legend()
        plt.show()
        
        
# **🔗 Link Sliders & Text Boxes**
def link_slider_text(slider, text):
    slider.observe(lambda change: text.set_trait("value", change["new"]), names="value")
    text.observe(lambda change: slider.set_trait("value", change["new"]), names="value")

for s, t in [(J_init_pos_slider, J_init_pos_text), (J_init_neg_slider, J_init_neg_text), 
             (J0_slider, J0_text), (Rs_slider, Rs_text), (n_slider, n_text), (Rsh_slider, Rsh_text)]:
    link_slider_text(s, t)
    
for slider in [J_init_pos_slider, J_init_neg_slider, 
               J0_slider, Rs_slider, n_slider, Rsh_slider]:
    slider.observe(update_plot, names="value")  # Update plot when slider changes


# **🔁 Optimize Button**
def optimize_parameters(_):
    with status_output:
        status_output.clear_output()
        print("Computing... Please wait.")

    result = minimize(cost_function, initial_params, args=(J_init_pos_slider.value, J_init_neg_slider.value), bounds=bounds, method='L-BFGS-B')
    
    J0_slider.value = result.x[0]
    Rs_slider.value = result.x[1] * 1000  # Convert back to Ω·cm² for UI
    n_slider.value = result.x[2]
    Rsh_slider.value = result.x[3] * 1000  # Convert back to Ω·cm² for UI

    with status_output:
        status_output.clear_output()
        print("Optimization Complete!")

    update_plot(None)

optimize_button = Button(description="Re-Optimize", button_style="info")
optimize_button.on_click(optimize_parameters)

# **📜 Layout**
ui = VBox([
    status_output,
    HBox([J_init_pos_slider, J_init_pos_text]),
    HBox([J_init_neg_slider, J_init_neg_text]),
    HBox([J0_slider, J0_text]),
    HBox([Rs_slider, Rs_text]),
    HBox([n_slider, n_text]),
    HBox([Rsh_slider, Rsh_text]),
    optimize_button,
    plot_output
])
display(ui)

update_plot(None)
